# Segmentation and Tracking using IceFloeTracker.jl

In [ ]:
# Setup environment
using Pkg
Pkg.add(;name="IceFloeTracker", rev="826/merge")
Pkg.add("Images")

In [ ]:
# Load packages
using IceFloeTracker: LopezAcosta2019Tiling, Watkins2025GitHub, FloeTracker, ChainedFilterFunction, MinimumWeightMatchingFunction
using Images: erode, segment_mean, labels_map, SegmentedImage, RGB, mosaicview
using Dates: DateTime

In [ ]:
data_loader = Watkins2025GitHub(; ref="a451cd5e62a10309a9640fbbe6b32a236fcebc70")
cases = data_loader(c -> c.case_number == 6)
mosaicview([c.modis_truecolor for c in cases], nrow=1)

In [ ]:
segmenter = LopezAcosta2019Tiling.Segment()
segmentation_results = [segmenter(c.modis_truecolor, c.modis_falsecolor, c.modis_landmask) for c in cases]

TODO: would one of these be a better interface?
- `segmentation_results = segmenter.(cases.modis_truecolor, cases.modis_falsecolor, cases.modis_landmask)`
- `segmentation_results = segmenter(cases.modis_truecolor, cases.modis_falsecolor, cases.modis_landmask)`

Would require:
- defining broadcasting for the Dataset struct
- defining broadcasting for the Segment interface

In [ ]:
tracker = FloeTracker(
    filter_function=ChainedFilterFunction(), 
    matching_function=MinimumWeightMatchingFunction()
)
tracking_results = tracker(segmentation_results, DateTime.(cases.metadata.start_date))

TODO: would one of these be a better interface?
- `tracking_results = tracker(segmentation_results, cases.metadata.passtime)`
  TODO: rename start_date to passtime in validation data set